# Creating A Collada Object - produces a basic wind turbine

The goal is to produce a basic Collada dae wind turbine object for display in Google Earth

In [1]:
from collada import *
import simplekml
import numpy as np

In this section, we outline how to create a collada document from scratch. First, let’s create an empy collada document:

In [2]:
mesh = Collada()

We could save this out, but it would be completely blank. Let’s first add a Material to the document:

In [3]:
mesh = Collada()
effect = material.Effect("effect0", [], "phong", diffuse=(1,0,0), specular=(0,1,0))
mat = material.Material("material0", "mymaterial", effect)
mesh.effects.append(effect)
mesh.materials.append(mat)

In [4]:
def create_cylinder(radius=5,length=150,sides=8):
    
    import math
    import numpy as np
    
    angle = 2*math.pi/sides

    vert_floats = np.empty([0,3])
    
    # create bottom cross-section
    for side in range(sides):
        #print(sector*angle)

        x = radius*math.cos(side*angle)
        y = radius*math.sin(side*angle)
        z = 0

        vert_floats = np.vstack([vert_floats,[x,y,z]])
        
    # create top cross-section
    for side in range(sides):
        #print(sector*angle)

        x = radius*math.cos(side*angle)
        y = radius*math.sin(side*angle)
        z = length

        vert_floats = np.vstack([vert_floats,[x,y,z]])

    #print(vert_floats)

    
    # define triangles
    indices = np.empty([0,6])
    
    for side in range(sides):
        tri1 = (0+side)%sides
        tri2 = (1+side)%sides
        tri3 = sides+side
        tri4 = sides+(side+1)%sides
        
        base_index = int(indices.size/2)
        
        indices = np.vstack([indices,
                              [tri1,base_index,tri2,base_index+1,tri3,base_index+2],
                              [tri2,base_index+3,tri3,base_index+4,tri4,base_index+5]
                             ])



    indices = indices.astype(int)
    #print(indices)

    return vert_floats,indices


In [5]:
def list_adder(lst,value):
    
    new_list = [item + value for item in lst]
    
    return new_list


In [6]:
def rotation_matrix(x_degrees=0,y_degrees=0,z_degrees=0):
    
    theta_x = np.radians(x_degrees)
    rx = np.array([[1,0,0],
                   [0,np.cos(theta_x),-np.sin(theta_x)],
                   [0,np.sin(theta_x), np.cos(theta_x)]])
    
    theta_y = np.radians(y_degrees)
    ry = np.array([[np.cos(theta_y),0,np.sin(theta_y)],
                   [0,1,0],
                   [-np.sin(theta_y),0, np.cos(theta_y)]])
    
    theta_z = np.radians(z_degrees)
    rz = np.array([[np.cos(theta_z),-np.sin(theta_z),0],
                   [np.sin(theta_z), np.cos(theta_z),0],
                   [0,0,1]])
    
    r = rx.dot(ry.dot(rz))
    
    return r

In [7]:
def create_blade(height=100,rotor_diamter=120,rotation=0):

    blade_vert_floats,blade_indices = create_cylinder(radius=2,length=rotor_diamter/2,sides=8)

    blade_vert_floats = blade_vert_floats.dot(rotation_matrix(x_degrees=rotation,y_degrees=0,z_degrees=0))

    blade_vert_floats[:,0] = blade_vert_floats[:,0]+25
    blade_vert_floats[:,2] = blade_vert_floats[:,2]+height+5
    
    return blade_vert_floats,blade_indices

In [8]:
height = 100
rotor_diamter = 120

# create tower
tower_vert_floats,tower_indices = create_cylinder(radius=5,length=height,sides=8)


# create nacelle
nacelle_vert_floats,nacelle_indices = create_cylinder(radius=8,length=50,sides=4)

nacelle_vert_floats = nacelle_vert_floats.dot(rotation_matrix(x_degrees=0,y_degrees=0,z_degrees=45))
nacelle_vert_floats = nacelle_vert_floats.dot(rotation_matrix(x_degrees=0,y_degrees=90,z_degrees=0))

nacelle_vert_floats[:,0] = nacelle_vert_floats[:,0]+20
nacelle_vert_floats[:,2] = nacelle_vert_floats[:,2]+height+5


# create blades
blade1_vert_floats,blade1_indices = create_blade(height=100,rotor_diamter=120,rotation=0)
blade2_vert_floats,blade2_indices = create_blade(height=100,rotor_diamter=120,rotation=120)
blade3_vert_floats,blade3_indices = create_blade(height=100,rotor_diamter=120,rotation=240)


# combine objects
vert_floats = np.vstack([tower_vert_floats,nacelle_vert_floats,blade1_vert_floats,blade2_vert_floats,blade3_vert_floats])

nacelle_indices = nacelle_indices + tower_vert_floats.size/3
blade1_indices = blade1_indices + (tower_vert_floats.size+nacelle_vert_floats.size)/3
blade2_indices = blade2_indices + (tower_vert_floats.size+nacelle_vert_floats.size+blade1_vert_floats.size)/3
blade3_indices = blade3_indices + (tower_vert_floats.size+nacelle_vert_floats.size+blade1_vert_floats.size+blade2_vert_floats.size)/3

indices = np.vstack([tower_indices,nacelle_indices,blade1_indices,blade2_indices,blade3_indices]).astype(int)



normal_floats = [0]*vert_floats.size*3

vert_src = source.FloatSource("cubeverts-array", vert_floats.flatten(), ('X', 'Y', 'Z'))
normal_src = source.FloatSource("cubenormals-array", np.array(normal_floats), ('X', 'Y', 'Z'))

geom = geometry.Geometry(mesh, "geometry0", "mycube", [vert_src, normal_src])

input_list = source.InputList()
input_list.addInput(0, 'VERTEX', "#cubeverts-array")
input_list.addInput(1, 'NORMAL', "#cubenormals-array")

triset = geom.createTriangleSet(indices.flatten(), input_list, "materialref")
geom.primitives.append(triset)
mesh.geometries.append(geom)

matnode = scene.MaterialNode("materialref", mat, inputs=[])
geomnode = scene.GeometryNode(geom, [matnode])
node = scene.Node("node0", children=[geomnode])

myscene = scene.Scene("myscene", [node])
mesh.scenes.append(myscene)
mesh.scene = myscene

mesh.write('tmp/collada03.dae')

In [9]:
tower_vert_floats

array([[ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.53553391e+00,  3.53553391e+00,  0.00000000e+00],
       [ 3.06161700e-16,  5.00000000e+00,  0.00000000e+00],
       [-3.53553391e+00,  3.53553391e+00,  0.00000000e+00],
       [-5.00000000e+00,  6.12323400e-16,  0.00000000e+00],
       [-3.53553391e+00, -3.53553391e+00,  0.00000000e+00],
       [-9.18485099e-16, -5.00000000e+00,  0.00000000e+00],
       [ 3.53553391e+00, -3.53553391e+00,  0.00000000e+00],
       [ 5.00000000e+00,  0.00000000e+00,  1.00000000e+02],
       [ 3.53553391e+00,  3.53553391e+00,  1.00000000e+02],
       [ 3.06161700e-16,  5.00000000e+00,  1.00000000e+02],
       [-3.53553391e+00,  3.53553391e+00,  1.00000000e+02],
       [-5.00000000e+00,  6.12323400e-16,  1.00000000e+02],
       [-3.53553391e+00, -3.53553391e+00,  1.00000000e+02],
       [-9.18485099e-16, -5.00000000e+00,  1.00000000e+02],
       [ 3.53553391e+00, -3.53553391e+00,  1.00000000e+02]])

In [10]:
nacelle_indices

array([[16., 16., 17., 17., 20., 18.],
       [17., 19., 20., 20., 21., 21.],
       [17., 22., 18., 23., 21., 24.],
       [18., 25., 21., 26., 22., 27.],
       [18., 28., 19., 29., 22., 30.],
       [19., 31., 22., 32., 23., 33.],
       [19., 34., 16., 35., 23., 36.],
       [16., 37., 23., 38., 20., 39.]])

In [11]:
tower_indices

array([[ 0,  0,  1,  1,  8,  2],
       [ 1,  3,  8,  4,  9,  5],
       [ 1,  6,  2,  7,  9,  8],
       [ 2,  9,  9, 10, 10, 11],
       [ 2, 12,  3, 13, 10, 14],
       [ 3, 15, 10, 16, 11, 17],
       [ 3, 18,  4, 19, 11, 20],
       [ 4, 21, 11, 22, 12, 23],
       [ 4, 24,  5, 25, 12, 26],
       [ 5, 27, 12, 28, 13, 29],
       [ 5, 30,  6, 31, 13, 32],
       [ 6, 33, 13, 34, 14, 35],
       [ 6, 36,  7, 37, 14, 38],
       [ 7, 39, 14, 40, 15, 41],
       [ 7, 42,  0, 43, 15, 44],
       [ 0, 45, 15, 46,  8, 47]])